In [1]:
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

import sys
sys.path.append('..')

import geometry_monkeypatch
from reach_tools import *

In [2]:
rogue = Reach.get_from_aw(1541)

In [24]:
json = rogue._download_raw_json_from_aw()
rapid_df = pd.DataFrame(json['CContainerViewJSON_view']['CRiverRapidsGadgetJSON_view-rapids']['rapids'])
rapid_df.head()

,approximate,deleted,description,difficulty,distance,isaccess,ishazard,isplayspot,isportage,isputin,istakeout,iswaterfall,name,photoid,rapidid,reachid,rlat,rloc,rlon,videoid
0,False,False,<p>\n\tJust downstream of the Grave Creek boat...,III,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grave Creek Riffle,887925,106035,1541,42.6500015258789,0101000020E610000021B0726891E55EC0333333333353...,-123.58699798584,0
1,False,False,<p>\n\tA vertical drop formed by a ledge. Foll...,III,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grave Creek Falls,889736,106036,1541,42.650002,0101000020E61000005531957EC2E55EC0D42AFA433353...,-123.589996,0
2,False,False,<p>\n\tA wide bedrock ledge blocks the river c...,IV+,1.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rainie Falls,16551,100131,1541,42.6500015258789,0101000020E61000008FC2F5285CE75EC0333333333353...,-123.61499786377,0
3,False,False,<p>\n\tNamed for Chinese miners who worked in ...,II,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China Gulch Rapids,0,106037,1541,42.6529998779297,0101000020E61000002B8716D9CEE75EC0DD2406819553...,-123.622001647949,0
4,False,False,<p>\n\tA long and wide gravel bar on the left ...,III+,4.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tyee Rapid,883978,100132,1541,42.6570014953613,0101000020E61000000AD7A3703DEA5EC06ABC74931854...,-123.660003662109,0


In [25]:
rapid_df.columns

Index(['approximate', 'deleted', 'description', 'difficulty', 'distance',
       'isaccess', 'ishazard', 'isplayspot', 'isportage', 'isputin',
       'istakeout', 'iswaterfall', 'name', 'photoid', 'rapidid', 'reachid',
       'rlat', 'rloc', 'rlon', 'videoid'],
      dtype='object')

In [28]:
rapid_df = rapid_df[['name', 'difficulty', 'distance', 'rlat', 'rlon']].copy()

In [62]:
waters = WATERS()

def get_waters_geometry(x, y):
    item = waters.get_epa_snap_point(x, y)
    return item['geometry']

r = rapid_df.iloc[0]
get_waters_geometry(r.rlon, r.rlat)

{'spatialReference': {'wkid': 4326},
 'x': -123.587026702054,
 'y': 42.6502221162582}

In [70]:
rapid_df.rlat = rapid_df.rlat.astype('float64')
rapid_df.rlon = rapid_df.rlon.astype('float64')

In [73]:
rapid_df = rapid_df[rapid_df['rlat'] != 0].copy()

In [74]:
rapid_waters = rapid_df.apply(lambda r: get_waters_geometry(r.rlon, r.rlat), axis=1)
rapid_waters

0     {'x': -123.587026702054, 'y': 42.6502221162582...
1     {'x': -123.590003764581, 'y': 42.6502966233287...
2     {'x': -123.615115370523, 'y': 42.6495826749051...
3     {'x': -123.62159363936, 'y': 42.6532986485857,...
4     {'x': -123.66013656485, 'y': 42.6571397256334,...
5     {'x': -123.666145088017, 'y': 42.6556455686817...
6     {'x': -123.675817427613, 'y': 42.6548059909584...
7     {'x': -123.678957476996, 'y': 42.6509672661847...
8     {'x': -123.685373847964, 'y': 42.647276458614,...
9     {'x': -123.703058529269, 'y': 42.6458846147827...
10    {'x': -123.706795119526, 'y': 42.6464120020166...
11    {'x': -123.71478469595, 'y': 42.6513666169945,...
12    {'x': -123.720996018647, 'y': 42.6529364552313...
13    {'x': -123.728943480337, 'y': 42.6522713638309...
14    {'x': -123.73063071901, 'y': 42.6530775294398,...
15    {'x': -123.733884593504, 'y': 42.6629918820222...
16    {'x': -123.735613949727, 'y': 42.6806939491507...
17    {'x': -123.744042870678, 'y': 42.685930996

In [75]:
rapid_df['SHAPE'] = rapid_waters

In [81]:
rapid_df.spatial.set_geometry('SHAPE')

In [87]:
rapid_df.spatial.to_featureclass('./rapids.shp')

'./rapids.shp'

In [91]:
print(rogue.takeout._geometry)

{'x': -124.049003601074, 'y': 42.6329002380371, 'spatialReference': {'wkid': 4326}}
